In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/data/기술과학분야_train_set.csv',index_col=0)
df_val = pd.read_csv('/content/drive/MyDrive/data/기술과학분야_valid_set.csv',index_col=0)

In [13]:
df['subdomain'].value_counts()

사회_의료     136303
경제일반      128234
IT        120052
국제기구      119967
빅데이터      119520
산업경제       80114
인공지능       79979
정치일반       64007
사회_환경      63877
국방외교       48055
북한         48002
문화_문화재     40033
문화_예술      39943
국제통상       32091
Name: subdomain, dtype: int64

In [3]:
df = df.drop(index=df[df['domain'] == '기후'].index)
df = df.reset_index(drop=True)

In [14]:
train = df
valid = df_val

In [15]:

# train = pd.concat([df.loc[df['domain']=='기술과학'].sample(150000),df.loc[df['domain']=='세계'].sample(150000),df.loc[df['domain']=='정치'].sample(150000),df.loc[df['domain']=='경제'].sample(150000)]).sample(600000).reset_index(drop=True)
# valid = pd.concat([df_val.loc[df_val['domain']=='기술과학'].sample(5000),df_val.loc[df_val['domain']=='세계'].sample(5000),df_val.loc[df_val['domain']=='정치'].sample(5000),df_val.loc[df_val['domain']=='경제'].sample(5000)]).sample(20000).reset_index(drop=True)

train['labels'] = train['subdomain'].replace(['국제통상', '국제기구', '국방외교', '빅데이터', '북한', '산업경제', '문화_예술', '경제일반','사회_환경', '문화_문화재', '정치일반', 'IT', '사회_의료', '인공지능'],[0, 1, 2, 3,4,5,6,7,8,9,10,11,12,13])
valid['labels'] = valid['subdomain'].replace(['국제통상', '국제기구', '국방외교', '빅데이터', '북한', '산업경제', '문화_예술', '경제일반','사회_환경', '문화_문화재', '정치일반', 'IT', '사회_의료', '인공지능'],[0, 1, 2, 3,4,5,6,7,8,9,10,11,12,13])

# 모델링

In [18]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.1 MB/s 
     |████████████████████████████████| 7.6 MB 56.5 MB/s 
     |████████████████████████████████| 163 kB 93.7 MB/s 


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained("klue/bert-base",num_labels=14)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
# 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
class CustomDataset(Dataset):

    def __init__(self,text, label,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.labels = label
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        text = str(self.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.float),
            }
    


training_set = CustomDataset(train['ko'],train['labels'], tokenizer, 128)
testing_set = CustomDataset(valid['ko'], valid['labels'],tokenizer, 128)

train_params = {'batch_size': 10,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 4,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [21]:
loss_fn = nn.CrossEntropyLoss() 

LEARNING_RATE = 5e-5
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)


train_steps = len(training_loader.dataset) // train_params['batch_size']
val_steps = len(testing_loader.dataset) // test_params['batch_size']


In [26]:
from tqdm import tqdm,tqdm_notebook
epochs=1
PATH = "/content/drive/MyDrive/model/"

for epoch in (range(epochs)):
    model.train()
    num_correct = 0
    trainning_loss = 0
    for i, data in enumerate(tqdm(training_loader)):
        
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        label = data['labels'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        
        output = model(ids, mask, token_type_ids)

        loss = loss_fn(output[0], label)
        trainning_loss += loss
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
          print("Training Losses: {}".format(trainning_loss/(i+1)))
          print("Trainong Accuracy:{}".format(num_correct/(i+1)))
    
        pred = output[0].argmax(dim=1)
        num_correct += torch.eq(pred,label).sum().float().item()
        


    print('EPOCH ', epoch+1)
    print("Training Losses: {}".format(trainning_loss/len(training_loader.dataset)))
    print("Trainong Accuracy:{}".format(num_correct/len(training_loader.dataset)))
    
    
    
    if epoch % 2 == 0 or epoch == epochs-1:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': trainning_loss,
                    }, PATH+f"model600000data{epoch}.pt")
    if epoch % 5 == 0:
        model.eval()
        num_correct = 0

        for i, data in enumerate(tqdm(testing_loader)):

              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              label = data['labels'].to(device, dtype = torch.long)


              optimizer.zero_grad()

              output = model(ids, mask, token_type_ids)
              loss = loss_fn(output[0], label)
              optimizer.step()

              pred = output[0].argmax(dim=1)
              num_correct += torch.eq(pred, label).sum().float().item()


        print('EPOCH ', epoch+1)
        print("validation Losses: {}".format(loss))
        print("validation Accuracy:{}".format(num_correct/len(testing_loader.dataset)))


  0%|          | 0/112018 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
  0%|          | 1/112018 [00:00<11:23:19,  2.73it/s]

Training Losses: 1.7634103298187256
Trainong Accuracy:0.0


  0%|          | 32/112018 [00:07<7:25:41,  4.19it/s]


KeyboardInterrupt: ignored

In [ ]:
model.eval()
num_correct = 0

for i, data in enumerate(tqdm(testing_loader)):

      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      label = data['labels'].to(device, dtype = torch.long)


      optimizer.zero_grad()

      output = model(ids, mask, token_type_ids)
      loss = loss_fn(output[0], label)
      optimizer.step()

      pred = output[0].argmax(dim=1)
      num_correct += torch.eq(pred, label).sum().float().item()


print('EPOCH ', epoch+1)
print("validation Losses: {}".format(loss))
print("validation Accuracy:{}".format(num_correct/len(testing_loader.dataset)))


# keyphrase classification


In [ ]:
import json

file_path = "/content/drive/MyDrive/data/keywords.json"

with open(file_path, 'r') as file:
    keywords = json.load(file)
    

In [ ]:
category = ['사회', '경제', '기술과학','정치']
keyword_params = {'batch_size': 5,
                'shuffle': True,
                'num_workers': 0
                }
for keyword in keywords:
  keyword_5 = list(keyword.values())[0]
  keyword_set = CustomDataset(keyword_5,[-1]*len(keyword_5), tokenizer, 128)
  keyword_loader = DataLoader(keyword_set, **keyword_params)
  for i, data in enumerate((keyword_loader)):

    ids = data['input_ids'].to(device, dtype = torch.long)
    mask = data['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    label = data['labels'].to(device, dtype = torch.long)

    output = model(ids, mask, token_type_ids)
    
    pred = pd.Series(output[0].argmax(dim=1).cpu().numpy()).replace([0,1,2,3],category)
    print([(keyword_5[i],pred[i]) for i in range(5)])

In [ ]:
category = ['세계', '경제', '기술과학','정치']
keyword = ["채권 양도 금지 특약.","주택난",'차별 금지법']
keyword_set = CustomDataset(keyword,[-1]*len(keyword), tokenizer, 128)
keyword_loader = DataLoader(keyword_set, **test_params)


for i, data in enumerate(tqdm(keyword_loader)):

    ids = data['input_ids'].to(device, dtype = torch.long)
    mask = data['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    label = data['labels'].to(device, dtype = torch.long)

    output = model(ids, mask, token_type_ids)

    pred = pd.Series(output[0].argmax(dim=1).cpu().numpy()).replace([0,1,2,3],['세계','경제','기술과학','정치'])

pred
